Install python libraries

In [1]:
import pandas as pd
import re
from spellchecker import SpellChecker
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer

[nltk_data] Downloading package stopwords to C:\Users\Abu
[nltk_data]     Hasan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Abu
[nltk_data]     Hasan\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to C:\Users\Abu
[nltk_data]     Hasan\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


Load dataset

In [2]:
df = pd.read_csv('./data/Products.csv', lineterminator='\n')

Preview Data

In [3]:
df.head(10)

,Unnamed: 0,id,product_name,category,product_description,price,location,url,page_id,create_time
0,0,243809c0-9cfc-4486-ad12-3b7a16605ba9,"Mirror wall art | in Wokingham, Berkshire | Gu...","Home & Garden / Dining, Living Room Furniture ...","Mirror wall art. Posted by Nisha in Dining, Li...",£5.00,"Wokingham, Berkshire",https://www.gumtree.com/p/mirrors-clocks-ornam...,1426704584,2022-02-26
1,1,1c58d3f9-8b93-47ea-9415-204fcc2a22e6,"Stainless Steel Food Steamer | in Inverness, H...",Home & Garden / Other Household Goods,Morphy Richard’s (model no 48755)Stainless ste...,£20.00,"Inverness, Highland",https://www.gumtree.com/p/other-household-good...,1426704579,2022-02-26
2,2,860673f1-57f6-47ba-8d2f-13f9e05b8f9a,"Sun loungers | in Skegness, Lincolnshire | Gum...",Home & Garden / Garden & Patio / Outdoor Setti...,I have 2 of these - collection only as I don’t...,£20.00,"Skegness, Lincolnshire",https://www.gumtree.com/p/outdoor-settings-fur...,1426704576,2022-02-26
3,3,59948726-29be-4b35-ade5-bb2fd7331856,Coffee side table from Ammunition ammo box hai...,"Home & Garden / Dining, Living Room Furniture ...",Great reclaimed army ammunition box used as co...,£115.00,"Radstock, Somerset",https://www.gumtree.com/p/other-dining-living-...,1426704575,2022-02-26
4,4,16dbc860-696e-4cda-93f6-4dd4926573fb,Modern Shannon Sofa for sale at low cost | in ...,"Home & Garden / Dining, Living Room Furniture ...",New Design Shannon Corner sofa 5 Seater Avail...,£450.00,"Delph, Manchester",https://www.gumtree.com/p/sofas/modern-shannon...,1426704570,2022-02-26
5,5,5707be4f-49d9-4feb-b2c8-aa0868405c65,"Spotlights | in Newent, Gloucestershire | Gumtree",Home & Garden / Other Household Goods,New in box not needed Spotlights . Posted by s...,£9.00,"Newent, Gloucestershire",https://www.gumtree.com/p/other-household-good...,1426704571,2022-02-26
6,6,d2244a7b-aef4-4495-aff7-0c55bf512d6e,💥💥White Wardrobe💥💥Brand new Wardrobe For sale ...,Home & Garden / Beds & Bedroom Furniture / War...,🌈🌈Call Dustin 0208-0049437\r🌈🌈Same Day Or any...,£419.00,"Huddersfield, West Yorkshire",https://www.gumtree.com/p/wardrobes-shelving-s...,1426704568,2022-02-26
7,7,485edc04-7bbc-430a-9430-f05a77f5a917,"Letter cage | in Newent, Gloucestershire | Gum...",Home & Garden / Other Household Goods,Selling for family member brand new too big fo...,£10.00,"Newent, Gloucestershire",https://www.gumtree.com/p/other-household-good...,1426704516,2022-02-26
8,8,7e538445-c75e-4802-8dc5-a9083a802278,"Double Pine Bed Frame | in Lytham St Annes, La...",Home & Garden / Beds & Bedroom Furniture / Dou...,Double Pine bed frame good condition. Reasonab...,£50.00,"Lytham St Annes, Lancashire",https://www.gumtree.com/p/double-beds/double-p...,1426704510,2022-02-26
9,9,1a64843d-72f3-4141-946f-fa9b978fe755,Solid pine dining table and chairs | in West K...,"Home & Garden / Dining, Living Room Furniture ...",Distressed solid pine table and 4 chairs In ex...,£200.00,"West Kilbride, North Ayrshire",https://www.gumtree.com/p/dining-tables-chairs...,1424517433,2022-02-26


Check shape (rows and columns)

In [4]:
df.shape

(7156, 10)

Check for nulls

In [5]:
df.isna().sum()

Unnamed: 0             0
id                     0
product_name           0
category               0
product_description    0
price                  0
location               0
url                    0
page_id                0
create_time            0
dtype: int64

Returning no nulls in data.

Notice how most product_names are in the form of title | location | website origin (Gumtree)

We want to preserve this across all rows but if we run:

In [6]:
df['product_name'].str.split('|', expand=True).count()

0     7156
1     7156
2     7156
3       24
4       18
5        5
6        2
7        2
8        1
9        1
10       1
11       1
12       1
13       1
14       1
dtype: int64

We get 14 columns, so that means some rows don't follow this convention.
Lets see how many rows have 3 columns (from product_name split) that end in 'Gumtree':

In [7]:
df['product_name'].str.split('|', expand=True)[2].value_counts()

 Gumtree                                                       7132
 VERONA CORNER SOFA OR 3+2 SEATER SOFA SET AVAILABLE STOCK        3
 in Croydon, London                                               3
 16GB Memory                                                      1
 in Bournemouth, Dorset                                           1
                                                                  1
 2021 Model                                                       1
 COUNTER TOP                                                      1
 Waterproof Liquid Eyeliner                                       1
 brand new                                                        1
 Excellent condition                                              1
 2022 MODEL                                                       1
 Money Problems                                                   1
                                                                  1
 Pedal                                          

Since >99% (7132/7156) rows follow the convention state above, we can drop the ones that don't.

Make new column for third column of product_name split by '|' :

In [8]:
df['gumtree'] = df['product_name'].str.split('|', expand=True)[2]

Drop rows where this column isn't Gumtree:

In [9]:
df = df[df['gumtree'].str.contains('Gumtree')]

Check unique values of this column, shpe and unique values of the split again to see if it worked:

In [10]:
df['gumtree'].value_counts()

 Gumtree    7132
Name: gumtree, dtype: int64

In [11]:
df.shape

(7132, 11)

In [12]:
df['product_name'].str.split('|', expand=True)[2].value_counts()

 Gumtree    7132
Name: 2, dtype: int64

Looks good, drop the column made for this purpose.

In [13]:
df.drop(['gumtree'], axis=1, inplace=True)

Now following the convention, we only need the title for product_name

In [14]:
df['product_name'] = df['product_name'].str.split('|', expand=True)[0]

In [15]:
df.head(10)

,Unnamed: 0,id,product_name,category,product_description,price,location,url,page_id,create_time
0,0,243809c0-9cfc-4486-ad12-3b7a16605ba9,Mirror wall art,"Home & Garden / Dining, Living Room Furniture ...","Mirror wall art. Posted by Nisha in Dining, Li...",£5.00,"Wokingham, Berkshire",https://www.gumtree.com/p/mirrors-clocks-ornam...,1426704584,2022-02-26
1,1,1c58d3f9-8b93-47ea-9415-204fcc2a22e6,Stainless Steel Food Steamer,Home & Garden / Other Household Goods,Morphy Richard’s (model no 48755)Stainless ste...,£20.00,"Inverness, Highland",https://www.gumtree.com/p/other-household-good...,1426704579,2022-02-26
2,2,860673f1-57f6-47ba-8d2f-13f9e05b8f9a,Sun loungers,Home & Garden / Garden & Patio / Outdoor Setti...,I have 2 of these - collection only as I don’t...,£20.00,"Skegness, Lincolnshire",https://www.gumtree.com/p/outdoor-settings-fur...,1426704576,2022-02-26
3,3,59948726-29be-4b35-ade5-bb2fd7331856,Coffee side table from Ammunition ammo box hai...,"Home & Garden / Dining, Living Room Furniture ...",Great reclaimed army ammunition box used as co...,£115.00,"Radstock, Somerset",https://www.gumtree.com/p/other-dining-living-...,1426704575,2022-02-26
4,4,16dbc860-696e-4cda-93f6-4dd4926573fb,Modern Shannon Sofa for sale at low cost,"Home & Garden / Dining, Living Room Furniture ...",New Design Shannon Corner sofa 5 Seater Avail...,£450.00,"Delph, Manchester",https://www.gumtree.com/p/sofas/modern-shannon...,1426704570,2022-02-26
5,5,5707be4f-49d9-4feb-b2c8-aa0868405c65,Spotlights,Home & Garden / Other Household Goods,New in box not needed Spotlights . Posted by s...,£9.00,"Newent, Gloucestershire",https://www.gumtree.com/p/other-household-good...,1426704571,2022-02-26
6,6,d2244a7b-aef4-4495-aff7-0c55bf512d6e,💥💥White Wardrobe💥💥Brand new Wardrobe For sale ...,Home & Garden / Beds & Bedroom Furniture / War...,🌈🌈Call Dustin 0208-0049437\r🌈🌈Same Day Or any...,£419.00,"Huddersfield, West Yorkshire",https://www.gumtree.com/p/wardrobes-shelving-s...,1426704568,2022-02-26
7,7,485edc04-7bbc-430a-9430-f05a77f5a917,Letter cage,Home & Garden / Other Household Goods,Selling for family member brand new too big fo...,£10.00,"Newent, Gloucestershire",https://www.gumtree.com/p/other-household-good...,1426704516,2022-02-26
8,8,7e538445-c75e-4802-8dc5-a9083a802278,Double Pine Bed Frame,Home & Garden / Beds & Bedroom Furniture / Dou...,Double Pine bed frame good condition. Reasonab...,£50.00,"Lytham St Annes, Lancashire",https://www.gumtree.com/p/double-beds/double-p...,1426704510,2022-02-26
9,9,1a64843d-72f3-4141-946f-fa9b978fe755,Solid pine dining table and chairs,"Home & Garden / Dining, Living Room Furniture ...",Distressed solid pine table and 4 chairs In ex...,£200.00,"West Kilbride, North Ayrshire",https://www.gumtree.com/p/dining-tables-chairs...,1424517433,2022-02-26


We also need to strip price of the pound sign

In [16]:
df['price'] = df['price'].apply(lambda x: x.replace('£', ''))

product_name, product_description and location need to go through text data cleaning.

The steps we will use:
1. Remove unwanted characters
2. Decapitalisation
3. Remove stop words
4. Lemmatising/Stemming
5. Vectorisation

In [17]:
def remove_unwanted_chars(df, columns):
    for column_name in columns:
        df['temporary_column'] = df[column_name].apply(lambda x: x.split())
        df['temporary_column'] = df['temporary_column'].apply(lambda x: [re.sub(r'[^0-9A-Za-z.+ ]', ' ', y) for y in x])
        df['temporary_column'] = df['temporary_column'].apply(lambda x: [y.replace('.', '') if y.find('.') == len(y)-1 else y for y in x])
        df[column_name] = df['temporary_column'].apply(lambda x: ' '.join(x))
        df.drop(['temporary_column'], axis=1, inplace=True)
    return df

In [18]:
df = remove_unwanted_chars(df, ['product_name', 'product_description', 'location'])

In [19]:
def decapitalise(df, columns):
    for column_name in columns:
        df[column_name] = df[column_name].apply(lambda x: x.lower())
    return df

In [20]:
df = decapitalise(df, ['product_name', 'product_description', 'location'])

In [21]:
def remove_stop_words(df, columns):
    stops = set(stopwords.words("english"))
    for column_name in columns:
        df['temporary_column'] = df[column_name].apply(lambda x: x.split())
        df['temporary_column'] = df['temporary_column'].apply(lambda x: [y for y in x if y not in stops])
        df[column_name] = df['temporary_column'].apply(lambda x: ' '.join(x))
        df.drop(['temporary_column'], axis=1, inplace=True)
    return df

In [22]:
df = remove_stop_words(df, ['product_name', 'product_description', 'location'])

In [23]:
def lemmatise_text(df, columns):
    lemmatizer = nltk.stem.WordNetLemmatizer()
    for column_name in columns:
        df['temporary_column'] = df[column_name].apply(lambda x: x.split())
        df['temporary_column'] = df['temporary_column'].apply(lambda x: [lemmatizer.lemmatize(y) for y in x])
        df[column_name] = df['temporary_column'].apply(lambda x: ' '.join(x))
        df.drop(['temporary_column'], axis=1, inplace=True)
    return df


In [24]:
df = lemmatise_text(df, ['product_name', 'product_description', 'location'])

The text data cleaning is almost done. Just one more step of vectorising the text data.

In [25]:
def vectorise_columns(df, column):
    v = TfidfVectorizer()
    x = v.fit_transform(df[column])
    temp_df = pd.DataFrame(x.toarray(), columns=v.get_feature_names_out())
    return temp_df

In [26]:
df_v_pn = vectorise_columns(df, 'product_name')
df_v_pd = vectorise_columns(df, 'product_description')
df_v_l = vectorise_columns(df, 'location')
df_vectorised = pd.concat([df.reset_index(drop=True), df_v_pn.reset_index(drop=True), df_v_pd.reset_index(drop=True), df_v_l.reset_index(drop=True)], axis=1)

Check:

In [27]:
df_check = df_vectorised.head(1)

In [28]:
non_zero_df = df_check.loc[:, (df_check != 0).any(axis=0)]

In [29]:
non_zero_df.head()

,id,product_name,category,product_description,price,location,url,page_id,create_time,art,...,living,mirror,nisha,ornament,posted,room,wall,wokingham,berkshire,wokingham
0,243809c0-9cfc-4486-ad12-3b7a16605ba9,mirror wall art,"Home & Garden / Dining, Living Room Furniture ...",mirror wall art posted nisha dining living roo...,5.00,wokingham berkshire,https://www.gumtree.com/p/mirrors-clocks-ornam...,1426704584,2022-02-26,0.627376,...,0.216113,0.42074,0.302557,0.248307,0.126775,0.181878,0.189711,0.295882,0.583263,0.812284


In [30]:
df.tail()

,Unnamed: 0,id,product_name,category,product_description,price,location,url,page_id,create_time
7151,7151,c4148656-78a9-4f3e-b393-134fdc5ef900,sony playstation vr move bundle,Video Games & Consoles / Consoles / PS4 (Sony ...,sony playstation vr move bundle353cash collect...,260.00,acocks green west midland,https://www.gumtree.com/p/ps4/sony-playstation...,1422159237,2022-02-28
7152,7152,564e3411-768d-4250-a624-b119d696f103,playstation vr v2 bundle,Video Games & Consoles / Consoles / PS4 (Sony ...,playstation vr v2 bundle355cash collection del...,235.00,acocks green west midland,https://www.gumtree.com/p/ps4/playstation-vr-v...,1422159464,2022-02-28
7153,7153,2b0a652b-46a2-4297-b619-5efeeb222787,oculus quest 2 256gb,Video Games & Consoles / Other Video Games & C...,pick 250comes two pistol stock gun game protec...,250.00,montrose angus,https://www.gumtree.com/p/other-video-games/oc...,1426668818,2022-02-28
7154,7154,719fd40a-870e-4144-b324-55dff2e66fb4,logitech driving force shifter,Video Games & Consoles / Video Game Accessorie...,bought christmas curry retailing 43.99 used al...,30.00,carrickfergus county antrim,https://www.gumtree.com/p/other-video-game-acc...,1426699715,2022-02-28
7155,7155,86d1806b-5575-4a7e-9160-f24f12be6c95,20 game nintendo switch bundle,Video Games & Consoles / Games,nintendo switch console used 2 3 time almost 3...,450.00,poole dorset,https://www.gumtree.com/p/video-games/20-games...,1426699649,2022-02-28
